## **letters**

In [6]:
import sqlite3

rom_letters = 'qwertyuiopasdfghjklzxcvbnm'
rom_letters += 'QWERTyuiOPasdfghjklzxcvbnm'.upper()
han_letters = 'ㅂㅈㄷㄱㅅㅛㅕㅑㅐㅔㅁㄴㅇㄹㅎㅗㅓㅏㅣㅋㅌㅊㅍㅠㅜㅡ'
han_letters += 'ㅃㅉㄸㄲㅆㅛㅕㅑㅒㅖㅁㄴㅇㄹㅎㅗㅓㅏㅣㅋㅌㅊㅍㅠㅜㅡ'
han_type = 'cccccvvvvvcccccvvvvccccvvv' * 2

con = sqlite3.connect('hangul.db')
cur = con.cursor()
cur.execute('DROP TABLE IF EXISTS letters')
cur.execute("""
    CREATE TABLE letters
    (
        key    VARCHAR(1)  NOT NULL PRIMARY KEY,
        hangul VARCHAR(1)  NOT NULL,
        type   VARCHAR(10) NOT NULL
    )
""")
for r, h, t in sorted(zip(rom_letters, han_letters, han_type), key=lambda x: (x[1], -ord(x[0]))):
    t = 'consonant' if t == 'c' else 'vowel'
    cur.execute('INSERT INTO letters (key, hangul, type) VALUES (?, ?, ?)', (r, h, t))
con.commit()
con.close()

## **merge_table**

In [50]:
for i in range(ord('가'), ord('하') + 1, 21 * 28):
    print(chr(i), end='')
print()
for i in range(ord('가'), ord('까'), 28):
    print(chr(i), end='')
print()
for i in range(ord('가'), ord('개')):
    print(chr(i), end='')

가까나다따라마바빠사싸아자짜차카타파하
가개갸걔거게겨계고과괘괴교구궈궤귀규그긔기
가각갂갃간갅갆갇갈갉갊갋갌갍갎갏감갑값갓갔강갖갗갘같갚갛

In [51]:
# ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ
initials = list('rRseEfaqQtTdwWczxvg')
# ㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣ
medials = 'k,o,i,O,j,p,u,P,h,hk,ho,hl,y,n,nj,np,nl,b,m,ml,l'.split(',')
# _ㄱㄲㄳㄴㄵㄶㄷㄹㄺㄻㄼㄽㄾㄿㅀㅁㅂㅄㅅㅆㅇㅈㅊㅋㅌㅍㅎ
finals = ',r,R,rt,s,sw,sg,e,f,fr,fa,fq,ft,fx,fv,fg,a,q,qt,t,T,d,w,c,z,x,v,g'.split(',')

keys = []
for i in initials:
    for m in medials:
        for f in finals:
            keys.append(i + m + f)

hanrom = {chr(i + ord('가')): c for i, c in enumerate(keys)}
romhan = {v: k for k, v in hanrom.items()}

len(hanrom), len(romhan)

(11172, 11172)

In [52]:
hr_keymap = {h: r for h, r in zip('ㅂㅈㄷㄱㅅㅛㅕㅑㅐㅔㅁㄴㅇㄹㅎㅗㅓㅏㅣㅋㅌㅊㅍㅠㅜㅡㅃㅉㄸㄲㅆㅒㅖ',
                                  'qwertyuiopasdfghjklzxcvbnmQWERTOP')}
rh_keymap = {r: h for h, r in hr_keymap.items()}

hr_false = {h: r for h, r in zip('ㅘ ㅙ ㅚ ㅝ ㅞ ㅟ ㅢ ㄳ ㄵ ㄶ ㄺ ㄻ ㄼ ㄽ ㄾ ㄿ ㅀ ㅄ'.split(),
                                 'hk ho hl nj np nl ml rt sw sg fr fa fq ft fx fv fg qt'.split())}
rh_false = {r: h for h, r in hr_false.items()}

hanrom.update(hr_false)
hanrom.update(hr_keymap)
romhan.update(rh_false)
romhan.update(rh_keymap)

len(hanrom), len(romhan)

(11223, 11223)

In [53]:
addmap = []

for han, rom in sorted(hanrom.items()):
    if len(rom) == 1:
        continue

    r1, r2 = rom[:-1], rom[-1]
    h1, h2 = romhan.get(r1), romhan.get(r2)
    # if h1 is None or h2 is None:
    #     print(r1, r2, h1, h2, han, rom)

    addmap.append((h1, h2, han))

len(addmap)

11190

In [54]:
import sqlite3

con = sqlite3.connect('hangul.db')
cur = con.cursor()
cur.execute('DROP TABLE IF EXISTS add_table')
cur.execute("""
    CREATE TABLE merge_table
    (
        h1     VARCHAR(1) NOT NULL,
        h2     VARCHAR(1) NOT NULL,
        hangul VARCHAR(1) NOT NULL UNIQUE,
        PRIMARY KEY (h1, h2)
    )
""")
for h1, h2, han in addmap:
    cur.execute('INSERT INTO merge_table (h1, h2, hangul) VALUES (?, ?, ?)', (h1, h2, han))
con.commit()
con.close()